# Details of the BioDendro pipeline

**This is technical manual of each step in BioDendo and the information you can extract. Please use the quick start for example, execution and data interrogation of results.**

## Running the pipeline manually

BioDendro essentially does the following steps:

1. Parse the MGF and components files
   - Optionally scaling and filtering Ions from MGF with a minimum intensity threshold.
2. Find Ions matching components
3. Bins mz values by proximity
4. Uses binned mz values to cluster components into a tree

In [1]:
# Load required modules

import os
import copy
import plotly
from BioDendro import preprocess
from BioDendro import cluster

Loading the MGF file is relatively straight forward using python.

The pipeline changes the MGF record titles in a way that won't always be portable.
If you encounter issues with record titles, you may wish to exclude the `split_msms_title` step.

In [2]:
# Load the MSMS records

with open("Fireflies_MSMS.mgf") as handle:
    mgf = preprocess.MGF.parse(handle)

mgf

MGF(records=[
MGFRecord(title='Ppyr_hemolymph_extract.173.173.1 File:"Ppyr_hemolymph_extract.raw", NativeID:"controllerType=0 controllerNumber=1 scan=173"', retention=58.5319032, pepmass=Ion(mz=118.08655413452, intensity=7337849.5), charge='1+', ions=[Ion(mz=55.05483998, intensity=1038985.0), Ion(mz=58.06570893, intensity=599806.5), Ion(mz=59.07383792, intensity=617962.375), Ion(mz=72.08117517, intensity=28507728.0), Ion(mz=118.0867653, intensity=13105900.0)]),
MGFRecord(title='Ppyr_hemolymph_extract.337.337.1 File:"Ppyr_hemolymph_extract.raw", NativeID:"controllerType=0 controllerNumber=1 scan=337"', retention=112.213572, pepmass=Ion(mz=120.081038697928, intensity=11558862.0), charge='1+', ions=[Ion(mz=56.05011642, intensity=144165.71875), Ion(mz=61.04036552, intensity=31103.697265625), Ion(mz=74.06097629, intensity=415685.0), Ion(mz=91.05498543, intensity=128501.1953125), Ion(mz=93.07017855, intensity=519245.96875), Ion(mz=102.0552512, intensity=153893.03125), Ion(mz=103.0548364, int

At this point we might decide to scale and/or filter the Ions for each MGFRecord.

In [3]:
with open("Fireflies_MSMS.mgf") as handle:
    mgf_scaled = preprocess.MGF.parse(handle, scaling=True, filtering=True, eps=0.2)
mgf_scaled

MGF(records=[
MGFRecord(title='Ppyr_hemolymph_extract.173.173.1 File:"Ppyr_hemolymph_extract.raw", NativeID:"controllerType=0 controllerNumber=1 scan=173"', retention=58.5319032, pepmass=Ion(mz=118.08655413452, intensity=7337849.5), charge='1+', ions=[Ion(mz=72.08117517, intensity=1.0), Ion(mz=118.0867653, intensity=0.4597314805304723)]),
MGFRecord(title='Ppyr_hemolymph_extract.337.337.1 File:"Ppyr_hemolymph_extract.raw", NativeID:"controllerType=0 controllerNumber=1 scan=337"', retention=112.213572, pepmass=Ion(mz=120.081038697928, intensity=11558862.0), charge='1+', ions=[Ion(mz=103.0548364, intensity=0.20429361231041457), Ion(mz=120.0804451, intensity=1.0)]),
MGFRecord(title='Ppyr_hemolymph_extract.673.673. File:"Ppyr_hemolymph_extract.raw", NativeID:"controllerType=0 controllerNumber=1 scan=673"', retention=223.69026, pepmass=Ion(mz=130.050003051758, intensity=4432127.0), charge='None', ions=[Ion(mz=84.04458615, intensity=1.0), Ion(mz=87.00419849, intensity=0.8858643370218536), Ion

You'll notice in the ions field of each MGF record that the "intensity" values have been scaled to numbers between 0 and 1, and that any ions with scaled intensity below 0.2 will be filtered out.

Loading components files is a similar process.

In [4]:
# Load the list of components to compare to the MSMS file

with open("./Fireflies_feature_list.txt") as handle:
    components = preprocess.SampleRecord.parse(handle)

components[:5]

[SampleRecord(mz=171.076385498046, retention=49.100565, original='Ppyr_hemolymph_extract_171.076385498046_0.81834275'),
 SampleRecord(mz=234.098556518554, retention=786.9699, original='Ppyr_hemolymph_extract_234.098556518554_13.116165'),
 SampleRecord(mz=234.098358154296, retention=538.098924, original='Ppyr_hemolymph_extract_234.098358154296_8.9683154'),
 SampleRecord(mz=252.10888671875, retention=407.356224, original='Ppyr_hemolymph_extract_252.10888671875_6.7892704'),
 SampleRecord(mz=313.150604248046, retention=51.691315200000005, original='Ppyr_hemolymph_extract_313.150604248046_0.86152192')]

Next we find the ions for each components.

In [5]:
# Remove redundant records with mass and retention time tolerance 

df = preprocess.remove_redundancy(components, mgf, mz_tol=0.002, retention_tol=5, neutral=False)
df.head()

,component,sample,mz
0,Ppyr_hemolymph_extract_501.27082824707_3.6575774,"Ppyr_hemolymph_extract.653.653.1 File:""Ppyr_he...",50.002351
1,Ppyr_hemolymph_extract_491.227661132812_10.204906,"Ppyr_hemolymph_extract.1829.1829.1 File:""Ppyr_...",50.015094
2,Ppyr_hemolymph_extract_431.178680419921_25.394594,"Ppyr_hemolymph_extract.4529.4529.1 File:""Ppyr_...",50.242590
3,Ppyr_hemolymph_extract_332.279602050781_12.443386,"Ppyr_hemolymph_extract.2229.2229.1 File:""Ppyr_...",50.334601
4,Ppyr_hemolymph_extract_328.915313720703_27.337551,"Ppyr_hemolymph_extract.4885.4885.1 File:""Ppyr_...",50.393199


And finally, we can cluster the data using a `Tree` object.

The interface to `Tree` is similar to `scikit-learn` objects, where first you initialise a model with parameters, then you fit it with some data.

In [6]:
# Using the non-redundant dataframe, bin analytes on threshold=0.004 and return a data matrix
# Cluster the data matrix using clustering_method="jaccard"
# Set threshold to color dendrogram and output clusters at cutoff=0.4

tree = cluster.Tree(threshold=0.004, clustering_method="braycurtis", cutoff=0.6)
tree.fit(df)

The threshold is the parameter that affects binning.
Increasing the threshold will reduce the number of bins, decreasing the threshold will make bins more granular.

The tree object now contains all of our intermediate data.

We can look at which samples have which bins in a [Pandas](https://pandas.pydata.org/) dataframe.
This is the matrix used to compute distances based on presence absence.

In [7]:
# One hot encoded matrix output from mz binning.
tree.onehot_df.head()

bins,100.0237_100.0224_100.0244,100.0389_100.0389_100.0389,100.0761_100.0757_100.0766,100.1127_100.1127_100.1127,100.8624_100.8624_100.8624,100.9323_100.9317_100.9326,100.9383_100.9383_100.9383,100.9523_100.9523_100.9523,101.0597_101.0590_101.0605,101.0963_101.0963_101.0963,...,99.0807_99.0806_99.0808,99.0919_99.0918_99.0919,99.1171_99.1170_99.1171,99.9243_99.9242_99.9243,99.9485_99.9485_99.9485,99.9691_99.9691_99.9691,99.9874_99.9874_99.9874,990.1105_990.1105_990.1105,992.7504_992.7504_992.7504,992.7560_992.7560_992.7560
component,,,,,,,,,,,,,,,,,,,,,
Ppyr_hemolymph_extract_1061.80706787109_24.011575,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Ppyr_hemolymph_extract_1065.4677734375_15.366083,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Ppyr_hemolymph_extract_1078.83374023437_24.011575,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Ppyr_hemolymph_extract_1082.49475097656_15.123002,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Ppyr_hemolymph_extract_1082.49487304687_21.307386,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


NB. "one-hot" encoding is a common technique in ML/statistice where 1 (or True) denotes presence and 0 (or False) denotes absence.
It's related to the concept of Dummy coding.

We can also look at the clusters derived from the computed tree.

In [8]:
# Clusters derived from the tree using cutoff=0.6
tree.clusters

array([121, 117, 121, 116, 114, 117, 115, 114, 124, 115, 116,  20,  33,
        42, 239, 265, 265, 187, 132, 182, 142, 140, 139, 210, 240, 122,
       266, 266, 266, 266, 232, 171, 226, 226, 273,  69, 253,  87,  87,
        73, 272, 272, 272, 272, 272, 173, 222, 222, 222, 144, 197, 259,
       172,  80,  88, 271, 271, 229,  51,  72,  72,  72,  72,  72, 269,
       262, 270, 267, 177, 176, 176, 254, 173, 255, 221, 221, 221, 221,
       221,  73, 257, 175, 143,  55,  52, 268, 123,  83,  54, 226, 274,
       243, 247,  78, 263, 263, 263, 263, 263, 263, 263, 263, 263, 263,
       263, 264, 262, 261, 261, 261, 227,  63,  54, 260, 260, 275, 258,
       217, 216, 179, 179, 179, 179, 179, 133,  82,  53, 245, 256, 200,
       200,  63,  59,  85, 150, 162, 211, 106, 156, 133, 241, 218, 218,
        81, 166, 249, 179,  46,  90, 233, 238,  59, 163, 211, 251, 252,
       158, 209, 220, 220, 129, 250, 128, 128,   6, 110,  35,  91,  44,
       151, 234, 233,  39, 208, 276, 236,  74, 142, 204, 106, 10

This is a [numpy](https://www.numpy.org/) array.
The order of these is the same as the row index in the one-hot dataframe.

So we can obtain clusters for each sample like so...

In [9]:
dict(zip(tree.onehot_df.index, tree.clusters))['Ppyr_hemolymph_extract_1082.49517822265_19.965233']

115

In fact we already store this dictionary in the tree object

In [10]:
# return the cluster number for which the queried analyte belongs
tree.cluster_map["Ppyr_hemolymph_extract_1082.49517822265_19.965233"]

115

As with the python pipeline in the quick-start notebook, we can easily choose a new cutoff for clustering.
In fact, the pipeline function returns a tree.

Note that cut_tree mutates the tree object, so if you want to retain the original clusters, take a copy of the tree first.

In [11]:
# Show the number of clusters before adjustment
print("BEFORE: Cutoff:", tree.cutoff, "n clusters:", len(set(tree.clusters)))

# Re-set a new cutoff for clusters
new_tree=copy.deepcopy(tree)
new_tree.cut_tree(cutoff=0.8)

# Show number of clusters after adjustment
print("AFTER: Cutoff:", new_tree.cutoff, "n clusters:", len(set(new_tree.clusters)))

BEFORE: Cutoff: 0.6 n clusters: 281
AFTER: Cutoff: 0.8 n clusters: 169


The `tree` and `new_tree` objects can now be compared side by side as per the quick start example. For now we will continue with the `new_tree`.

To get the cluster numbers along-side the presence absence data we can simply add a new colum to the dataframe, 
since it's already in the correct order.

In [12]:
# Make sure you take a copy to avoid editing the data.
oh = tree.onehot_df.copy()
oh["cluster"] = tree.clusters

# reorder columns
oh = oh[["cluster"] + [col for col in oh.columns if col != "cluster"]]

# Sort values by cluster
oh.sort_values(by="cluster", inplace=True)

oh.head()

bins,cluster,100.0237_100.0224_100.0244,100.0389_100.0389_100.0389,100.0761_100.0757_100.0766,100.1127_100.1127_100.1127,100.8624_100.8624_100.8624,100.9323_100.9317_100.9326,100.9383_100.9383_100.9383,100.9523_100.9523_100.9523,101.0597_101.0590_101.0605,...,99.0807_99.0806_99.0808,99.0919_99.0918_99.0919,99.1171_99.1170_99.1171,99.9243_99.9242_99.9243,99.9485_99.9485_99.9485,99.9691_99.9691_99.9691,99.9874_99.9874_99.9874,990.1105_990.1105_990.1105,992.7504_992.7504_992.7504,992.7560_992.7560_992.7560
component,,,,,,,,,,,,,,,,,,,,,
Ppyr_hemolymph_extract_358.148544311523_14.967814,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Ppyr_hemolymph_extract_330.11734008789_10.945718,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Ppyr_hemolymph_extract_854.381774902343_16.613713,2,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Ppyr_hemolymph_extract_615.263275146484_10.945718,3,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Ppyr_hemolymph_extract_503.210754394531_6.7892704,3,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


We can quickly extract the subtable for clusters or samples using the `cluster_table` method.

In [13]:
# for visualising the ion table of your cluster of interest

# NB .head just shows the first few rows of the table, used here to keep things tidy.
# Remove .head() if you want to see the whole table.
new_tree.cluster_table(cluster=3).head()

bins,119.0860_119.0851_119.0864,131.0859_131.0847_131.0897,143.0845_143.0812_143.0864,159.0807_159.0796_159.0813,183.1169_183.1168_183.1170,197.0358_197.0355_197.0361,215.0468_215.0468_215.0468,223.0521_223.0507_223.0530,226.9866_226.9862_226.9869,227.0485_227.0485_227.0485,...,689.2396_689.2396_689.2397,751.2865_751.2862_751.2868,767.2951_767.2948_767.2955,79.0547_79.0540_79.0553,83.0497_83.0490_83.0498,86.0969_86.0964_86.0974,869.3800_869.3795_869.3805,885.3728_885.3727_885.3729,91.0554_91.0536_91.0584,92.7557_92.7526_92.7585
component,,,,,,,,,,,,,,,,,,,,,
Ppyr_hemolymph_extract_869.372192382812_26.122142,True,False,True,False,False,False,False,True,True,False,...,False,True,False,False,True,False,True,False,True,True
Ppyr_hemolymph_extract_885.366485595703_24.318087,True,False,False,False,False,True,True,False,False,False,...,True,False,True,True,True,False,False,True,True,False
Ppyr_hemolymph_extract_886.399169921875_26.122142,True,False,True,True,True,True,False,True,True,False,...,False,True,False,True,True,True,True,False,True,True
Ppyr_hemolymph_extract_902.393859863281_24.318087,True,True,True,True,False,True,False,False,False,True,...,True,False,True,True,True,False,False,True,True,False


In [14]:
new_tree.cluster_table(sample="Ppyr_hemolymph_extract_1082.49517822265_19.965233").head()

bins,1029.4494_1029.4494_1029.4494,1029.4552_1029.4549_1029.4555,1047.4338_1047.4338_1047.4338,1047.4380_1047.4380_1047.4380,1047.4646_1047.4646_1047.4646,1047.4686_1047.4686_1047.4686,105.0701_105.0698_105.0707,107.0492_107.0489_107.0498,1082.4910_1082.4910_1082.4910,119.0860_119.0851_119.0864,...,95.0494_95.0491_95.0501,951.4002_951.4002_951.4002,951.4058_951.4046_951.4070,969.4141_969.4141_969.4141,969.4192_969.4192_969.4192,969.4337_969.4337_969.4337,969.4393_969.4393_969.4393,987.4268_987.4268_987.4268,987.4319_987.4311_987.4327,987.4369_987.4369_987.4369
component,,,,,,,,,,,,,,,,,,,,,
Ppyr_hemolymph_extract_1082.49487304687_21.307386,True,False,False,False,False,True,True,True,False,True,...,True,False,True,False,False,False,True,False,True,False
Ppyr_hemolymph_extract_1082.49517822265_19.965233,False,True,True,False,False,False,True,False,False,False,...,False,False,False,False,True,False,False,False,False,True
Ppyr_hemolymph_extract_1082.49530029296_18.991166,False,False,False,False,True,False,True,False,True,True,...,False,True,False,False,False,True,False,True,False,False
Ppyr_hemolymph_extract_1100.50646972656_18.35508,False,True,False,True,False,False,True,False,False,False,...,False,False,True,True,False,False,False,False,True,False


We can write out the results of the clustering as we saw in the quick-start notebook.
Note that the `path` will be a directory where multiple files are written, and it must already exist.

Also, any results already in the directory will be overwritten.
The default path will use a combination of the date and time to avoid overwriting data.

In [15]:
# Create the directory to store results.
# Note that this will fail if the directory already exists.
# You can change exist_ok to True, but the files from old results will be overwritten or remain unchanged.
os.makedirs("results-cutoff-08", exist_ok=False)

# Generate the plots of clusters.
new_tree.write_summaries(path="results-cutoff-08")

To remove a directory and its files you can type.

```python
import shutil
shutil.rmtree("results-cutoff-08")
```

To list the created files...

In [16]:
os.listdir("results-cutoff-08")[:10]

['cluster_24_1.xlsx',
 'cluster_87_1.xlsx',
 'cluster_104_1.xlsx',
 'cluster_108_2.png',
 'cluster_149_1.png',
 'cluster_92_1.png',
 'cluster_70_1.xlsx',
 'clusters.xlsx',
 'cluster_142_2.png',
 'cluster_8_3.xlsx']

We can inspect the hierarchical clustering linkage map output from scipy.
Unfortunately, this lacks a bit of intuition.
The first two columns refer to two nodes in the tree.
The third column gives the distance between the two nodes.
And the fourth column gives the number of leaves that are in or beneath these nodes.

In [17]:
# Linkage tree from scipy.
new_tree.tree[:6]

array([[4.00000000e+01, 4.10000000e+01, 0.00000000e+00, 2.00000000e+00],
       [4.95000000e+02, 4.96000000e+02, 0.00000000e+00, 2.00000000e+00],
       [1.79000000e+02, 1.80000000e+02, 0.00000000e+00, 2.00000000e+00],
       [4.34000000e+02, 4.35000000e+02, 0.00000000e+00, 2.00000000e+00],
       [2.93000000e+02, 2.94000000e+02, 4.00000000e-02, 2.00000000e+00],
       [5.08000000e+02, 5.09000000e+02, 5.26315789e-02, 2.00000000e+00]])

The tree is probably more useful to you.
You can write an interactive tree to a file using the `plot` method on the `Tree` object.

In [18]:
# To write out the tree.
# NB the results folder already exists from the previous step where we wrote summaries.
new_tree.plot(filename="results-cutoff-08/simple_dendrogram.html", width=1000, height=1000);

Note that the width and height values are in pixels.
If your sample names are fairly long like ours are, you might like to fiddle with the height.
You can navigate to the .html file and open it in your favourite browser.

Since you're already using a Jupyter notebook you might like to plot the tree inline with your other analysis.
We can do that using some plotly functionality.

The `plot` method returns a Plotly `Figure` object which you can use with normal plotly functions.
So we can use the `notebook_mode` to show it inline.

In [19]:
# Make it a bit smaller this time
iplot = new_tree.plot(width=1000, height=1000)

# for visualising plot inline
# You only need to set connected to True once per notebook,
# but doing it multiple times won't hurt.
plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot(iplot)

Rolling over the leaf nodes with your mouse you'll be able to see which cluster each component belongs to, so you can then track it down in your results folder.

The `plot` method is just a wrapper around the `dendrogram` function in `BioDendro.plot.dendrogram`.
If you want to customise your plot e.g. with custom titles, you can do that there.

The `dendrogram` function is a modified version of plotlys [`figure_factory.create_dendrogram`](https://plot.ly/python/dendrogram/) function.
It is modified to reduce complexity, to handle text-rollover and to allow us to provide precomputed trees rather than having to recompute the tree each time (possibly yielding different trees and different clusters).

In [20]:
from BioDendro.plot import dendrogram

iplot = dendrogram(tree, width=800, height=1000, title="TreesRCool", xlabel="Samples", ylabel="Distance")
plotly.offline.iplot(iplot)

There are some features that we're working on.

In dendrogram, the `hovertext` parameter allows you to specify arbitrary text for when you hover over the data.
However, the current tree drawing algorithm reorders the internal nodes, and we can't find the correct order for internal node data from the output of that function.
For now know that it exists, and if you figure it out, please let us know!